In [1]:
import numpy as np
from math import atan2

import k3d

#import param
import panel as pn
from panel.interact import interact
from panel import widgets

pn.extension('katex',
             #raw_css = [""" .bk.bk-input-group label.bk span.bk { color: white; }""" ]  # dark theme is not well supported  
            ) #, 'mathjax')

<div style="float:center;width:100%;text-align:center;"><strong style="height:100px;color:darkred;font-size:40px;">Quadric Equations</strong></div>

In [2]:
class QuadricPlot:
    def eqn_text( a, b, c ):
        '''Latex Representation of the Quadric'''
        _a  = np.round(a,2); a_txt = f'{_a}' + '\\ x^2'  if _a != 0. else ''
        _b  = np.round(b,2); b_txt = f'{_b}' + '\\ x y'  if _b != 0. else ''
        _c  = np.round(c,2); c_txt = f'{_c}' + '\\ y^2'  if _c != 0. else ''
        txt = "$\\Large{ f(x,y) \\ = \\ " + \
              a_txt + ( ' + ' if _b > 0. and a != 0. else '' ) + b_txt \
                    + ( ' + ' if _c > 0. else '' ) + c_txt + " }$"
        return txt

    def __init__(self, N=256, e=2. ):
        x             = np.linspace(-e, e, N, dtype=np.float32); y = x
        self.x,self.y = np.meshgrid(x,y)
        self.e        = e
        self.a        = 0.
        self.b        = 1.
        self.c        = 0.
        self.cut      = 0.
        self._k3d_plot()
        self.text     = QuadricPlot.eqn_text( self.a, self.b, self.c )

    def _eigen_decomposition(self):
        # ------------- eigendecomposition
        A = np.array([[     self.a,  0.5*self.b ],
                      [ 0.5*self.b,      self.c ]])
        e_vals,e_vectors = np.linalg.eigh(A)

        e_vals = np.round(e_vals,2)
        theta  = np.round( 180./np.pi*atan2( e_vectors[1,0], e_vectors[0,0]))

        return e_vals, e_vectors, theta

    def _surface_and_eigenvectors(self):
        z = self.a*self.x**2 + self.b*self.x*self.y + self.c*self.y**2
        p = 0*self.x + self.cut

        e_vals, e_vectors, theta = self._eigen_decomposition()

        l = 1.05*np.sqrt(2.)*self.e
        q_vecs = { 'origin': [0,0,self.cut,  0,0,self.cut],
                   'delta':  [l*e_vectors[0,0],l*e_vectors[1,0],0,
                              l*e_vectors[0,1],l*e_vectors[1,1],0, ]}
        return e_vals, theta, z,p,q_vecs

    def _k3d_plot(self):
        plot = k3d.plot()
        plot.background_color=0x39c6d8 #0x000055
        
        self.e_vals, self.theta, z,p, self.q_vecs = self._surface_and_eigenvectors()
        self.q_vecs['colors'] = [0x520000,0x520000,  0x520000,0x520000]

        self.surface = k3d.surface( z, xmin=-self.e,xmax=self.e, ymin=-self.e,ymax=self.e, color=0xffffff)
        self.plane   = k3d.surface( p, xmin=-self.e,xmax=self.e, ymin=-self.e,ymax=self.e, color=0xc3d3d9)
        self.q       = k3d.vectors( self.q_vecs['origin'], self.q_vecs['delta'], colors=self.q_vecs['colors'], line_width=0.2, use_head=False)

        plot       += self.surface
        plot       += self.plane
        plot       += self.q

        plot.camera_auto_fit = True
        plot.grid_auto_fit   = True

        self.plot = plot

    def update_shape(self,a,b,c,cut):
        self.a = a; self.b=b; self.c = c; self.cut = cut
        self.e_vals, self.theta, z,p, q_vecs = self._surface_and_eigenvectors()
        self.q_vecs['origin'] = q_vecs['origin']
        self.q_vecs['delta' ] = q_vecs[ 'delta']
        self.surface.heights  = z
        self.plane.heights    = p
        self.q.origins        = self.q_vecs['origin']
        self.q.vectors        = self.q_vecs['delta']
        self.text             = QuadricPlot.eqn_text( self.a, self.b, self.c )


    def update_camera(self, radial_distance,phi,height):
        rad = phi*np.pi/180.
        self.plot.camera = [radial_distance*np.sin(rad),radial_distance*np.cos(rad), height,
                            0,0,0,
                            0,0,1]

    def layout(self):
        v = pn.Row( pn.pane.Pane( self.plot, height=500, width=500, margin=15),
            pn.pane.LaTeX( self.text, renderer='katex' ),
        )
        return v

In [3]:
E           = 2.
quadric     = QuadricPlot(e=E)

plot_layout = quadric.layout()
#pn.Row( pn.pane.Pane(quadric.plot, height=500, width=500),
#                     pn.pane.LaTeX(quadric.text, renderer='katex' ),
#              )

def update_shape( q, a, b, c, cut ):
    q.update_shape(a,b,c,cut)
    plot_layout[1].object = q.text

shape_ctrls = interact( lambda a,b,c,cut: update_shape(quadric, a,b,c,cut),
          a   = widgets.FloatSlider( name = 'a',   start= -E, end= E, step=.01, value=0.),
          b   = widgets.FloatSlider( name = 'b',   start= -E, end= E, step=.01, value=1.),
          c   = widgets.FloatSlider( name = 'c',   start= -E, end= E, step=.01, value=0.),
          cut = widgets.FloatSlider( name = 'cut', start=-20, end=20, step=.1,  value=0.),
        )

camera_ctrls = interact( lambda radial_distance, phi, hgt : quadric.update_camera(radial_distance,phi,hgt),
                         radial_distance = widgets.FloatSlider( name = 'radial_distance',  start=    0., end=500, step=1.,  value= 3.),
                         phi             = widgets.FloatSlider( name = 'phi',              start=    0., end=360, step=0.5, value=30.),
                         hgt             = widgets.FloatSlider( name = 'height',           start=    0., end=500, step=1.,  value= 5.),
                       )
#print( plot_layout )
#print( shape_ctrls )
#print( camera_ctrls )

left_pane  = plot_layout[0]
right_pane = pn.Column( pn.Spacer(height=5), plot_layout[1], pn.Spacer(height=5),
                        shape_ctrls[0], pn.Spacer(height=20),
                        pn.widgets.StaticText(value='<strong style="color:darkred;font-size:20px;">Camera Controls</strong>'),
                        camera_ctrls[0]
              )

#######################################################################
# REMINDER: currently this only works in Jupyter classic
#######################################################################
pn.Row( left_pane, pn.Spacer( width = 10 ), right_pane ).servable()

Row
    [0] IPyWidget(Plot, height=500, margin=15, sizing_mode='fixed', width=500)
    [1] Spacer(width=10)
    [2] Column
        [0] Spacer(height=5)
        [1] LaTeX(str, renderer='katex')
        [2] Spacer(height=5)
        [3] Column
            [0] FloatSlider(end=2.0, name='a', start=-2.0, step=0.01)
            [1] FloatSlider(end=2.0, name='b', start=-2.0, step=0.01, value=1.0)
            [2] FloatSlider(end=2.0, name='c', start=-2.0, step=0.01)
            [3] FloatSlider(end=20, name='cut', start=-20)
        [4] Spacer(height=20)
        [5] StaticText(value='<strong style="color:dark...)
        [6] Column
            [0] FloatSlider(end=500, name='radial_distance', step=1.0, value=3.0)
            [1] FloatSlider(end=360, name='phi', step=0.5, value=30.0)
            [2] FloatSlider(end=500, name='height', step=1.0, value=5.0)